<a href="https://colab.research.google.com/github/ShayanRiyaz/Rice-Crop-Disease-Detection/blob/master/rice-crop-disease-detection-using-tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Rice Fields](https://github.com/ShayanRiyaz/Rice-Crop-Disease-Detection/blob/master/images/rice-fields.png?raw=1)

# <div align="center">Rice Crop Disease Detection using TensorFlow</div>
<img src="https://github.com/ShayanRiyaz/Rice-Crop-Disease-Detection/blob/master/images/TF.png?raw=1" align = "right" width = "200">



## Table of Contents
- [Importing Libraries](#ImportingLibraries)
- [Loading Dataset](#LoadingDataset)
- [Resizing Image](#Resize)
- [Split into Training and Validation](#Split)
- [Image Count](#ImageCount)
- [Viewing Images](#ViewingImages)
    - [BrownSpot](#BrownSpot)
    - [Healthy](#Healthy)
    - [Hispa](#Hispa)
    - [LeafBlast](#LeafBlast)
- [Data Augmentation and Generators](#DataAugAndGen)
- [Callback](#Callback)
- [Models](#Models)
    - [1. Model - Conv2D](#Conv2D)
        - [Metrics](#MetricsConv2D)
        - [Observing the Convolutions](#ObservingConv2D)
    - [2. Model - InceptionV3](#InceptionV3)
        - [Metrics](#MetricInceptionv3)
    - [3. Model - EfficientNet](#EfficientNet)
        - [Metrics](#MetricsEfficientv2)
- [Export as TensorFlow LITE](#TFLITE)

# Importing Libraries <a name="ImportingLibraries"></a>

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import tensorflow_hub as hub
import os
import PIL
import cv2


from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from keras import optimizers
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    


Number of replicas: 1


In [5]:
print("Numpy==",np.__version__)
print("tensorflow==", tf.__version__)
print("pandas==",pd.__version__)
print("PIL==",PIL.__version__)
print('matplotlib==',matplotlib.__version__)
print('cv2==',cv2.__version__)

print("tensorflow_hub==",hub.__version__)
print("Eager mode:",tf.executing_eagerly())
print("GPU is","available" if tf.test.is_gpu_available() else"Not Available")


Numpy== 1.18.5
tensorflow== 2.3.0
pandas== 1.0.5
PIL== 7.0.0
matplotlib== 3.2.2
cv2== 4.1.2
tensorflow_hub== 0.8.0
Eager mode: True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is Not Available


# Loading Dataset <a name="LoadingDataset"></a>

In [ ]:
#!/usr/bin/python

import os, sys

# Create new Train and val folders

base_dir = 'kaggle/input/RiceLeafs'
train_path = '/kaggle/input/RiceLeafs/train'
val_path = 'kaggle/input/RiceLeafs/validation/'

column_names = os.listdir(train_path)
for i in column_names:
    os.makedirs(f'../kaggle/output/train/{i}')
    os.makedirs(f'../kaggle/output/validation/{i}')

out_path = '../kaggle/output/train/'




# Resizing Image <a name="Resize"></a>

In [ ]:
from PIL import Image
def resize(input_path,folder,column_name):
    dirs = os.listdir(input_path)
    for item in dirs:
        item_path = input_path +'/' +item
        if os.path.isfile(item_path):
            #print('CHECK')
            im = Image.open(item_path)

            # Check whether the specified 
            # path exists or not 
            outpath = f'/kaggle/kaggle/output/{folder}/{column_name}'
            temp_out_path = outpath+'/'+item
            f, e = os.path.splitext(temp_out_path)

            imResize = im.resize((255,255), Image.ANTIALIAS)
            #print('CHECK 3')
            imResize.save(f + '.jpg', 'JPEG', quality=90)





In [ ]:
input_path = '../input/RiceLeafs/train/Healthy'
folder = 'train'
column_name = 'Healthy'
resize(input_path,folder,column_name)

input_path = '../input/RiceLeafs/train/BrownSpot'
folder = 'train'
column_name = 'BrownSpot'
resize(input_path,folder,column_name)

input_path = '../input/RiceLeafs/train/Hispa'
folder = 'train'
column_name = 'Hispa'
resize(input_path,folder,column_name)

input_path = '../input/RiceLeafs/train/LeafBlast'
folder = 'train'
column_name = 'LeafBlast'
resize(input_path,folder,column_name)

print('Done with train resizing')

In [ ]:
## VALIDATION
input_path = '../input/RiceLeafs/validation/Healthy'
folder = 'validation'
column_name = 'Healthy'
resize(input_path,folder,column_name)

input_path = '../input/RiceLeafs/validation/BrownSpot'
folder = 'validation'
column_name = 'BrownSpot'
resize(input_path,folder,column_name)

input_path = '../input/RiceLeafs/validation/Hispa'
folder = 'validation'
column_name = 'Hispa'
resize(input_path,folder,column_name)

input_path = '../input/RiceLeafs/validation/LeafBlast'
folder = 'validation'
column_name = 'LeafBlast'
resize(input_path,folder,column_name)

print('Done with Validation resizing')

In [ ]:
os.path.exists('/kaggle/kaggle/output/validation/Healthy/')

In [ ]:
os.path.exists('/kaggle/kaggle/output/train/')
os.path.exists('/kaggle/kaggle/output/validation/')


In [ ]:
os.listdir('/kaggle/kaggle/output/train/BrownSpot/')

In [ ]:
data_dir = os.path.join(os.path.dirname('/kaggle/kaggle/'), 'output')

# Split into Training and Validation  <a name="Split"></a>

In [ ]:
# Use this if you avoided the resizing
data_dir = os.path.join(os.path.dirname('/output/'), 'RiceLeafs')

In [ ]:
train_dir = os.path.join(data_dir, 'train')
train_BrownSpot_dir = os.path.join(train_dir, 'BrownSpot')
train_Healthy_dir = os.path.join(train_dir, 'Healthy')
train_Hispa_dir = os.path.join(train_dir, 'Hispa')
train_LeafBlast_dir = os.path.join(train_dir, 'LeafBlast')


validation_dir = os.path.join(data_dir, 'validation')
validation_BrownSpot_dir = os.path.join(validation_dir, 'BrownSpot')
validation_Healthy_dir = os.path.join(validation_dir, 'Healthy')
validation_Hispa_dir = os.path.join(validation_dir, 'Hispa')
validation_LeafBlast_dir = os.path.join(validation_dir, 'LeafBlast')

In [ ]:
train_BrownSpot_names = os.listdir(train_BrownSpot_dir)
print(train_BrownSpot_names[:10])

train_Healthy_names =  os.listdir(train_Healthy_dir)
print(train_Healthy_names[:10])

train_Hispa_names = os.listdir(train_Hispa_dir)
print(train_Hispa_names[:10])

train_LeafBlast_names =  os.listdir(train_LeafBlast_dir)
print(train_LeafBlast_names[:10])

## Image Count <a name="ImageCount"></a>

In [ ]:

import time
import os
from os.path import exists

def count(dir, counter=0):
    "returns number of files in dir and subdirs"
    for pack in os.walk(dir):
        for f in pack[2]:
            counter += 1
    return dir + " : " + str(counter) + " files"

print('total images for training :', count(train_dir))
print('total images for validation :', count(validation_dir))

## Viewing Images  <a name="ViewingImages"></a>

In [ ]:
# Parameters for our graph; we'll outpu images in a 4x4 configuration
nrows = 4
ncols = 4

# for iternating over images
pic_index = 0

### BrownSpot <a name="BrownSpot"></a>

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics

fig = plt.gcf()
fig.set_size_inches(ncols *4, nrows*4)

pic_index += 8
next_BrownSpot_pix = [os.path.join(train_BrownSpot_dir, fname)
                for fname in train_BrownSpot_names[pic_index-8:pic_index]]
for i, img_path in enumerate(next_BrownSpot_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows,ncols,i +1)
  #sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

### Healthy <a name="Healthy"></a>

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics

fig = plt.gcf()
fig.set_size_inches(ncols *4, nrows*4)

pic_index += 8
next_Healthy_pix = [os.path.join(train_Healthy_dir, fname)
                for fname in train_Healthy_names[pic_index-8:pic_index]]


for i, img_path in enumerate(next_Healthy_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows,ncols,i +1)
  #sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

### Hispa <a name="Hispa"></a>

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics

fig = plt.gcf()
fig.set_size_inches(ncols *4, nrows*4)

pic_index += 8

next_Hispa_pix = [os.path.join(train_Hispa_dir, fname)
                for fname in train_Hispa_names[pic_index-8:pic_index]]


for i, img_path in enumerate(next_Hispa_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows,ncols,i +1)
  #sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

### LeafBlast <a name="LeafBlast"></a>

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics

fig = plt.gcf()
fig.set_size_inches(ncols *4, nrows*4)

pic_index += 8

next_LeafBlast_pix = [os.path.join(train_LeafBlast_dir, fname)
                for fname in train_LeafBlast_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_LeafBlast_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows,ncols,i +1)
  #sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

# Data Augmentation and Generators <a name="DataAugAndGen"></a>

In [ ]:
IMAGE_SHAPE = (244, 244)
BATCH_SIZE = 64 #@param {type:"integer"}

In [ ]:
# Inputs are suitably resized for the selected module. Dataset augmentation (i.e., random distortions of an image each time it is read) improves training, esp. when fine-tuning.

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE)

do_data_augmentation = True #@param {type:"boolean"}
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      fill_mode='nearest' )
else:
  train_datagen = validation_datagen
  
train_generator = train_datagen.flow_from_directory(
    train_dir,  
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE)

In [ ]:
train_generator.num_classes

## Callback <a name="Callback"></a>

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,log = {}):
    if(log.get('accuracy')> 0.99):
      if(log.get('val_accuracy')>0.99):
        print("\n Reached 99% Accuracy for both train and val.")
        self.model.stop_training = True

callbacks = MyCallback()

# Models <a name="Model"></a>

### 1. Model - Conv2D <a name="Conv2D"></a>

In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',input_shape = (244,244,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4,activation = 'softmax')

],    name = 'Conv2D_Model')

model.summary()

In [ ]:
LEARNING_RATE = 0.001 #@param {type:"number"}

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
EPOCHS=10 #@param {type:"integer"}

history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples//train_generator.batch_size,
        epochs=EPOCHS,
        validation_data=validation_generator,
        callbacks = [callbacks],
        validation_steps=validation_generator.samples//validation_generator.batch_size)

### Metrics <a name="MetricsConv2D"></a>

In [ ]:
import matplotlib.pylab as plt
import numpy as np

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.show()

#### Observing the Convolutions  <a name="ObservingConv2D"></a>

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img


# Lets define a new Model that will take an image as an input and will output 
# the intermediate representations for all layers in the previous model after 
# the first

successive_outputs = [layer.output for layer in model.layers[1:]]

# Visualization_model = Model(img_input,successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input,
                                            outputs = successive_outputs)

# Lets prepare a random input image form the training set.

BrownSpot_img_files = [os.path.join(train_BrownSpot_dir, f) for f in train_BrownSpot_names]
Healthy_files = [os.path.join(train_Healthy_dir, f) for f in train_Healthy_names]
img_path = random.choice(BrownSpot_img_files + Healthy_files)


img = load_img(img_path,target_size = (244,244)) # This is a PIL image
x = img_to_array(img)  # Numpy array with shape (244,244,3)
x = x.reshape((1,) + x.shape) # Numpy array with shape (1,244,244,3)

# Rescale by 1/255
x /=255


# Let's run our image through our network, thus obtaining all
# Intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers so we can have them as part of our plot
layer_names = [layer.name for layer in model.layers[1:]]


# Now lets display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv/maxpool layers, for the fully-connected layers
    n_features = feature_map.shape[-1] # number of features in feature map
    # The feature map has shape (1,size,size,n_features)
    size = feature_map.shape[1]
    # We will title our images in this matrix
    display_grid = np.zeros((size, size* n_features))
    for i in range(n_features):
      # Post process the feature to make it visibly palatable
      x = feature_map[0,:,:,i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x+= 128
      x = np.clip(x,0,255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:,i*size:(i+1)*size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale*n_features,scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid,aspect = 'auto', cmap = 'viridis')

### 2.Model - Inception <a name="InceptionV3"></a>

#### Downloading Weights <a name="DownloadWeights"></a>

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf.dim_ordering_tf_kernels.notop.h5

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
local_weights_file = '/tmp/inception_v3_weights_tf.dim_ordering_tf_kernels.notop.h5'

pre_trained_model = InceptionV3(
                                input_shape = (244,244,3),
                                include_top= False,
                                weights = None
)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
    


In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print(f'The shape of the last layer is {last_layer.output_shape}')
output_layer = last_layer.output

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

x = tf.keras.layers.Flatten()(output_layer)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
#x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(4, activation='softmax')(x)

model = Model(pre_trained_model.input, x,name="RiceLeafs_Inception_model")

In [ ]:
LEARNING_RATE = 0.001 #@param {type:"number"}

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = LEARNING_RATE),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
EPOCHS=10 #@param {type:"integer"}

history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples//train_generator.batch_size,
        epochs=EPOCHS,
        validation_data=validation_generator,
        callbacks = [callbacks],
        validation_steps=validation_generator.samples//validation_generator.batch_size)

#### Metrics <a name="MetricsInceptionv3"></a>

In [ ]:
import matplotlib.pylab as plt
import numpy as np

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.show()

### 3. Model - EfficientNet v2 <a name="EfficientNet"></a>

#### TensorFlow Hub Dataset
- [EfficientNet B7](https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
model = tf.keras.Sequential([
hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),

  tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.build([None, 244, 244, 3])

In [ ]:
#Compile model specifying the optimizer learning rate

LEARNING_RATE = 0.0001 #@param {type:"number"}

model.compile(
   optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
   loss='categorical_crossentropy',
   metrics=['accuracy'])

In [ ]:
EPOCHS=10 #@param {type:"integer"}

history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples//train_generator.batch_size,
        epochs=EPOCHS,
        validation_data=validation_generator,
        #callbacks = [callbacks],
        validation_steps=validation_generator.samples//validation_generator.batch_size)

### Metrics <a name="MetricsEfficientv2"></a>

In [ ]:
import matplotlib.pylab as plt
import numpy as np

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.show()

# Predict

In [ ]:
IMAGE_SHAPE[0]

In [ ]:
# Import OpenCV
import cv2

# Utility
import itertools
import random
from collections import Counter
from glob import iglob


def load_image(filename):
    img = cv2.imread(os.path.join(data_dir, validation_dir, filename))
    img = cv2.resize(img,(IMAGE_SHAPE[0], IMAGE_SHAPE[1]) )
    img = img /255
    
    return img


def predict(image):
    probabilities = model.predict(np.asarray([img]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}

In [ ]:
for idx, filename in enumerate(random.sample(validation_generator.filenames, 5)):
    print("SOURCE: class: %s, file: %s" % (os.path.split(filename)[0], filename))
    
    img = load_image(filename)
    prediction = predict(img)
    print("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))
    plt.imshow(img)
    plt.figure(idx)    
    plt.show()

# Export as TensorFlowLITE <a name="TFLITE"></a>

In [ ]:
import time
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
tf.keras.experimental.export_saved_model(model, export_path)

export_path

In [ ]:
# Now confirm that we can reload it, and it still gives the same results
reloaded = tf.keras.experimental.load_from_saved_model(export_path, custom_objects={'KerasLayer':hub.KerasLayer}) # custom_objects depends on model

In [ ]:
# convert the model to TFLite
!mkdir "tflite_models"
TFLITE_MODEL = "tflite_models/rice_leaf_disease.tflite"


# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : reloaded(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

# Convert the model to standard TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)